In [37]:
import tensorflow as tf
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt

In [38]:
image_dir = "./clothes_data"
model_dir = "./models"

In [39]:
feature_list = list(pd.read_csv("feature_list.csv")["name"])
print(feature_list)
brand_list = list(pd.read_csv("brand_list.csv")["brand"])
print(brand_list)

['Shirt', 'T-Shirt', '上半身 Top', '下半身 Bottom', '休閒褲/裙 casual pants/skirt', '吊帶褲/裙 overalls/jumper skirt', '外套 Jacket', '工裝褲/裙 cargo pants/skirt', '毛衣、針織衣 Sweater', '洋裝 Dress', '牛仔材質 Denim', '牛仔褲/裙 jeans / demi skirt', '百褶裙 pleated skirt', '網紗 Mesh', '緊身褲 leggings', '羽絨衣 Down coat / jacket', '背心 Vest', '裙子 skirt', '褲子 pants', '西裝 Suit', '西裝褲、裙 dress pants、skirt', '連帽衣 Hoodie', '運動材質 Sport Tops', '運動褲/裙 sweat pants / sports skirt']
['50 percent', 'AIR SPACE', 'Adidas', 'Caco', 'GAP', 'H&M', 'Lativ', 'Net', 'New Balance', 'Nike', 'PAZZO', 'Puma', 'Queen Shop', 'Timberland', 'Under Armour', 'Uniqlo']


In [40]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array

img_height, img_width = 180, 180


def load_and_preprocess_image(filepath):
    img = load_img(filepath, target_size=(img_height, img_width))
    img_array = img_to_array(img) / 255.0
    return img_array

In [ ]:
from tensorflow.keras.models import load_model


def predict_one_feature(test_image, feature_id):
    try:
        id = 0
        loaded_model = load_model(
            os.path.join(model_dir, f"model_feature_{feature_id}_{id}.keras")
        )
        test_image = np.expand_dims(test_image, axis=0)
        prediction = loaded_model.predict(test_image, verbose=0)
        return 1 if prediction[0][0] > 0.5 else 0
    except Exception as e:
        print(e)
        return 0


def predict_brand(test_image):
    try:
        id = 0
        loaded_model = load_model(os.path.join(model_dir, f"model_brand_{id}.keras"))
        test_image = np.expand_dims(test_image, axis=0)
        prediction = loaded_model.predict(test_image, verbose=0)
        predicted_classes = np.argmax(prediction, axis=1)
        brand = brand_list[int(predicted_classes)]
        print("predict brand:", brand)
        return predicted_classes
    except Exception as e:
        print(e)
        pass


def predict_length(test_image):
    try:
        id = 0
        loaded_model = load_model(os.path.join(model_dir, f"model_length_{id}.keras"))
        test_image = np.expand_dims(test_image, axis=0)
        prediction = loaded_model.predict(test_image, verbose=0)
        return 1 if prediction[0][0] > 0.5 else 0
    except Exception as e:
        print(e)
        pass

In [42]:
def predict_all(test_img):
    predictions = {}
    brand = predict_brand(test_img)
    for i in range(len(brand_list)):
        predictions[f"品牌 Brand_{i}"] = 1 if brand == i else 0
    # predictions["brand"] = brand
    for i in range(len(feature_list)):
        feature = feature_list[i]
        pred = predict_one_feature(test_img, i)
        predictions[feature] = pred
    print("predictions:", predictions)
    predictions_df = pd.DataFrame([predictions])
    predictions_df = predictions_df[sorted(predictions_df.columns)]
    return predictions_df

In [43]:
from joblib import load
import pickle


def predict_price(classify_res, model_type="rf"):

    if model_type == "nn":
        loaded_model = load_model(os.path.join(model_dir, f"model_prediction.keras"))
        price = loaded_model.predict(classify_res, verbose=0)
    else:
        # Load the model using pickle
        with open("./models/random_forest_model.pkl", "rb") as f:
            loaded_model = pickle.load(f)
        loaded_model = load("./models/random_forest_model.joblib")
        price = loaded_model.predict(classify_res)
    return price

In [47]:
df = pd.read_csv("processed.csv")
df = df.loc[:, ~df.columns.str.contains("^Unnamed")]
df = df.drop(df.index[df.index % 10 != 0])
df["filepath"] = df["照片 Picture"].apply(lambda x: os.path.join(image_dir, f"{x}"))
test_num = 5
df = df.sample(n=test_num)

In [ ]:
mae_nn = 0
mae_rf = 0
for id, filepath, ans, brand in zip(
    df.index, df["filepath"], df["價格 Price"], df["品牌 Brand"]
):
    print(id, brand, filepath, ans)
    img = load_and_preprocess_image(filepath)
    # plt.imshow(img)
    pred = predict_all(img)
    price_nn = predict_price(pred, "nn")
    price_rf = predict_price(pred, "rf")
    print("price(nn, rf):", price_nn, price_rf)
    dif_nn = abs(price_nn - ans)
    dif_rf = abs(price_rf - ans)
    mae_nn += dif_nn
    mae_rf += dif_rf

mae_nn /= test_num
mae_rf /= test_num
print("mae(nn, rf):", mae_nn, mae_rf)

770 AIR SPACE ./clothes_data\image_00770.jpg 490


C:\Users\doo12\AppData\Local\Temp\ipykernel_15348\1499496932.py:25: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  brand = brand_list[int(predicted_classes)]


brand: AIR SPACE
File not found: filepath=./models\model_feature_8_0.keras. Please ensure the file is an accessible `.keras` zip file.
File not found: filepath=./models\model_feature_9_0.keras. Please ensure the file is an accessible `.keras` zip file.
File not found: filepath=./models\model_feature_10_0.keras. Please ensure the file is an accessible `.keras` zip file.
File not found: filepath=./models\model_feature_11_0.keras. Please ensure the file is an accessible `.keras` zip file.
File not found: filepath=./models\model_feature_16_0.keras. Please ensure the file is an accessible `.keras` zip file.
File not found: filepath=./models\model_feature_17_0.keras. Please ensure the file is an accessible `.keras` zip file.
File not found: filepath=./models\model_feature_18_0.keras. Please ensure the file is an accessible `.keras` zip file.
File not found: filepath=./models\model_feature_19_0.keras. Please ensure the file is an accessible `.keras` zip file.
File not found: filepath=./models

C:\Users\doo12\AppData\Local\Temp\ipykernel_15348\1499496932.py:25: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  brand = brand_list[int(predicted_classes)]


brand: Caco
File not found: filepath=./models\model_feature_8_0.keras. Please ensure the file is an accessible `.keras` zip file.
File not found: filepath=./models\model_feature_9_0.keras. Please ensure the file is an accessible `.keras` zip file.
File not found: filepath=./models\model_feature_10_0.keras. Please ensure the file is an accessible `.keras` zip file.
File not found: filepath=./models\model_feature_11_0.keras. Please ensure the file is an accessible `.keras` zip file.
File not found: filepath=./models\model_feature_16_0.keras. Please ensure the file is an accessible `.keras` zip file.
File not found: filepath=./models\model_feature_17_0.keras. Please ensure the file is an accessible `.keras` zip file.
File not found: filepath=./models\model_feature_18_0.keras. Please ensure the file is an accessible `.keras` zip file.
File not found: filepath=./models\model_feature_19_0.keras. Please ensure the file is an accessible `.keras` zip file.
File not found: filepath=./models\mode

...

brand: H&M
File not found: filepath=./models\model_feature_8_0.keras. Please ensure the file is an accessible `.keras` zip file.
File not found: filepath=./models\model_feature_9_0.keras. Please ensure the file is an accessible `.keras` zip file.
File not found: filepath=./models\model_feature_10_0.keras. Please ensure the file is an accessible `.keras` zip file.
File not found: filepath=./models\model_feature_11_0.keras. Please ensure the file is an accessible `.keras` zip file.
File not found: filepath=./models\model_feature_16_0.keras. Please ensure the file is an accessible `.keras` zip file.
File not found: filepath=./models\model_feature_17_0.keras. Please ensure the file is an accessible `.keras` zip file.
File not found: filepath=./models\model_feature_18_0.keras. Please ensure the file is an accessible `.keras` zip file.
File not found: filepath=./models\model_feature_19_0.keras. Please ensure the file is an accessible `.keras` zip file.
File not found: filepath=./models\model

C:\Users\doo12\AppData\Local\Temp\ipykernel_15348\1499496932.py:25: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  brand = brand_list[int(predicted_classes)]


brand: Under Armour
File not found: filepath=./models\model_feature_8_0.keras. Please ensure the file is an accessible `.keras` zip file.
File not found: filepath=./models\model_feature_9_0.keras. Please ensure the file is an accessible `.keras` zip file.
File not found: filepath=./models\model_feature_10_0.keras. Please ensure the file is an accessible `.keras` zip file.
File not found: filepath=./models\model_feature_11_0.keras. Please ensure the file is an accessible `.keras` zip file.
File not found: filepath=./models\model_feature_16_0.keras. Please ensure the file is an accessible `.keras` zip file.
File not found: filepath=./models\model_feature_17_0.keras. Please ensure the file is an accessible `.keras` zip file.
File not found: filepath=./models\model_feature_18_0.keras. Please ensure the file is an accessible `.keras` zip file.
File not found: filepath=./models\model_feature_19_0.keras. Please ensure the file is an accessible `.keras` zip file.
File not found: filepath=./mod

C:\Users\doo12\AppData\Local\Temp\ipykernel_15348\1499496932.py:25: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  brand = brand_list[int(predicted_classes)]


brand: H&M
File not found: filepath=./models\model_feature_8_0.keras. Please ensure the file is an accessible `.keras` zip file.
File not found: filepath=./models\model_feature_9_0.keras. Please ensure the file is an accessible `.keras` zip file.
File not found: filepath=./models\model_feature_10_0.keras. Please ensure the file is an accessible `.keras` zip file.
File not found: filepath=./models\model_feature_11_0.keras. Please ensure the file is an accessible `.keras` zip file.
File not found: filepath=./models\model_feature_16_0.keras. Please ensure the file is an accessible `.keras` zip file.
File not found: filepath=./models\model_feature_17_0.keras. Please ensure the file is an accessible `.keras` zip file.
File not found: filepath=./models\model_feature_18_0.keras. Please ensure the file is an accessible `.keras` zip file.
File not found: filepath=./models\model_feature_19_0.keras. Please ensure the file is an accessible `.keras` zip file.
File not found: filepath=./models\model